In [1]:
import sys;
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, neg, pairing, Z1, Z2
import galois
from functools import reduce
import numpy as np
from operator import add

GF = galois.GF(curve_order)

def evaluate_polynomial_galois(coefs, x, GF):
    result = GF(0)
    for coef in coefs:
        result = result * x + GF(coef)
    return int(result)

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

In [7]:
# x^3+x+5
# Ensemble des varibles = ['1', 'out', 'x', 'v1', 'v2', 'v3']
U=[
    [3648040478639879203707734290876212514758060733402672390616367364429301415937, 21888242871839275222246405745257275088548364400416034343698204186575808495612, 18240202393199396018538671454381062573790303667013361953081836822146507079690, 21888242871839275222246405745257275088548364400416034343698204186575808495612], 
    [0, 0, 0, 0], 
    [14592161914559516814830937163504850059032242933610689562465469457717205663744, 5, 7296080957279758407415468581752425029516121466805344781232734728858602831861, 8], 
    [10944121435919637611123202872628637544274182200208017171849102093287904247809, 21888242871839275222246405745257275088548364400416034343698204186575808495613, 10944121435919637611123202872628637544274182200208017171849102093287904247818, 21888242871839275222246405745257275088548364400416034343698204186575808495611], 
    [10944121435919637611123202872628637544274182200208017171849102093287904247808, 10944121435919637611123202872628637544274182200208017171849102093287904247812, 21888242871839275222246405745257275088548364400416034343698204186575808495610, 4], 
    [18240202393199396018538671454381062573790303667013361953081836822146507079681, 21888242871839275222246405745257275088548364400416034343698204186575808495616, 3648040478639879203707734290876212514758060733402672390616367364429301415938, 21888242871839275222246405745257275088548364400416034343698204186575808495616]]

V=[
    [7296080957279758407415468581752425029516121466805344781232734728858602831872, 10944121435919637611123202872628637544274182200208017171849102093287904247811, 3648040478639879203707734290876212514758060733402672390616367364429301415931, 3], 
    [0, 0, 0, 0], 
    [14592161914559516814830937163504850059032242933610689562465469457717205663745, 10944121435919637611123202872628637544274182200208017171849102093287904247806, 18240202393199396018538671454381062573790303667013361953081836822146507079686, 21888242871839275222246405745257275088548364400416034343698204186575808495615], 
    [0, 0, 0, 0], 
    [0, 0, 0, 0], 
    [0, 0, 0, 0]]

W=[
    [0, 0, 0, 0], 
    [18240202393199396018538671454381062573790303667013361953081836822146507079681, 21888242871839275222246405745257275088548364400416034343698204186575808495616, 3648040478639879203707734290876212514758060733402672390616367364429301415938, 21888242871839275222246405745257275088548364400416034343698204186575808495616], 
    [0, 0, 0, 0], 
    [3648040478639879203707734290876212514758060733402672390616367364429301415936, 10944121435919637611123202872628637544274182200208017171849102093287904247810, 7296080957279758407415468581752425029516121466805344781232734728858602831868, 4], 
    [10944121435919637611123202872628637544274182200208017171849102093287904247809, 21888242871839275222246405745257275088548364400416034343698204186575808495613, 10944121435919637611123202872628637544274182200208017171849102093287904247818, 21888242871839275222246405745257275088548364400416034343698204186575808495611], 
    [10944121435919637611123202872628637544274182200208017171849102093287904247808, 10944121435919637611123202872628637544274182200208017171849102093287904247812, 21888242871839275222246405745257275088548364400416034343698204186575808495610, 4]
]


a=[1, 135, 5, 25, 125, 130]

nb_eq = len(U[0])

#=================================Pour trouver t(x)  ==========================
values = [i for i in range(1, nb_eq+1)] # Car on a 6 colonnes

poly_t = galois.Poly([1], field=GF)  # Start with the polynomial '1' in the given field
for val in values:
    poly_t *= galois.Poly([1, -val], field=GF)
t=[]
for elem in poly_t.coeffs:
    t.append(int(elem))
#==============================================================================


tau=GF(13)

tau_G1=[]
for i in range(0, nb_eq):
    tau_G1.append(multiply(G1, int(tau**i)))

tau_G2=[]
for i in range(0, nb_eq):
    tau_G2.append(multiply(G2, int(tau**i)))

t_tau=evaluate_polynomial_galois(t, tau, GF)

tau_t_G1=[]
for i in range(0, nb_eq):
    tau_t_G1.append(multiply(multiply(G1, t_tau),int(tau**i)))

print("tau_G1=",tau_G1)
print("tau_G2=",tau_G2)
print("tau_t_G1=",tau_t_G1)

alpha=GF(5)
beta=GF(7)
gamma=GF(11)
delta=GF(13)

# alpha_G1=multiply(G1, int(alpha))
# beta_G1=multiply(G1, int(beta))
# beta_G2=multiply(G2, int(beta))
# gamma_G2=multiply(G2, int(gamma))
# delta_G1=multiply(G1, int(delta))
# delta_G2=multiply(G2, int(delta))

U_polys=[]
for elem in U:
    U_polys.append(galois.Poly((elem),field=GF))

V_polys=[]
for elem in V:
    V_polys.append(galois.Poly((elem),field=GF))

W_polys=[]
for elem in W:
    W_polys.append(galois.Poly((elem),field=GF))

witness = np.array(a) % curve_order

Ua = inner_product_polynomials_with_witness(U_polys, witness)
Va = inner_product_polynomials_with_witness(V_polys, witness)
Wa = inner_product_polynomials_with_witness(W_polys, witness)

t_poly=galois.Poly(t,field=GF)

h_quo = (Ua * Va - Wa) // t_poly
h_rem = (Ua * Va - Wa) % t_poly

assert h_rem == 0, "h_rem != 0"

# print(h_quo)
print("reste = ",h_rem)
print("h(x) = ",h_quo)

#Pour etre bien ZK, le prouver va choisir r et s aléatoirement
r=GF(30)
s=GF(200)
m = len(a)

A = alpha + reduce(add, [a[i] * (U_polys[i](tau)) for i in range(0, m)]) + r * delta
print("A=",A)

B = beta + reduce(add, [a[i] * (V_polys[i](tau)) for i in range(0, m)]) + s * delta
print("B=",B)

C = (reduce(add,[a[i]*(beta * (U_polys[i](tau)) + alpha * (V_polys[i](tau)) + W_polys[i](tau)) for i in range(2, m)]) + h_quo(tau) * t_poly(tau)) / delta + (A * s + B * r - r * s * delta)
print("C=",C)

A_1=multiply(G1,int(A)) 
C_1=multiply(G1,int(C))
B_2=multiply(G2,int(B))

lhs = pairing(B_2, A_1)
print("lhs=",lhs)

D = multiply(G1,int(reduce(add, [ a[i] * (beta * U_polys[i](tau) + alpha * V_polys[i](tau) + W_polys[i](tau)) for i in range(0, 2)]) / gamma))

rhs = pairing(multiply(G2,int(beta)),multiply(G1,int(alpha))) * pairing( multiply(G2,int(gamma)),D)* pairing(multiply(G2,int(delta)),C_1)
print("rhs=",rhs)
print(lhs == rhs)


tau_G1= [(1, 2), (2672242651313367459976336264061690128665099451055893690004467838496751824703, 18247534626997477790812670345925575171672701304065784723769023620148097699216), (12787339590274394701415274422371285504222742952103028517882107571519658525136, 4739954905410646502518070840395475123473740446172208598017525809409688232767), (11196867986020524887868151181298169377551861207909865023458178113072991099778, 14907424744721644019295601827429271389033647158022784104065095546615339379684)]
tau_G2= [((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531)), ((16137324789686743234629608741537369181251990815455155257427276976918350071287, 280672898440571232725436467950720547829638241593507531241322547969961007057), (1213642065022645

In [ ]:

#Le prouver gen r,s deux randoms pour avoir du ZKP
r=GF(30)
s=GF(200)

h_tau=evaluate_polynomial_galois(h_quo.coeffs, tau, GF)
ht_tau=h_tau*t_tau
print("ht(tau)=",ht_tau)

#===============================CALCUL DE A============================================
A_G1=None
A_0=alpha_G1

A_1=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_U_tau_G1=multiply(G1, eval_U_tau)
    eval_aU_tau_G1=multiply(eval_U_tau_G1, int(a[i]))
    A_1=add(A_1, eval_aU_tau_G1)

A_2=multiply(delta_G1, int(r))

A_G1=add(A_0, add(A_1, A_2))
print("A_G1=",A_G1)

#===============================CALCUL DE B_2============================================
B_G2=None
B_0=beta_G2

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G2=multiply(G2, eval_V_tau)
    eval_aV_tau_G2=multiply(eval_V_tau_G2, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G2)

B_2=multiply(delta_G2, int(s))

B_G2=add(B_0, add(B_1, B_2))
print("B_G2=",B_G2)

#===============================CALCUL DE B_1============================================
B_G1=None
B_0=beta_G1

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G1=multiply(G1, eval_V_tau)
    eval_aV_tau_G1=multiply(eval_V_tau_G1, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G1)

B_2=multiply(delta_G1, int(s))

B_G1=add(B_0, add(B_1, B_2))
print("B_G1=",B_G1)

#===============================CALCUL DE C ============================================
C=None

C_0=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_W_tau=evaluate_polynomial_galois(W[i], tau, GF)
    temp=int(beta)*int(eval_U_tau)+int(alpha)*int(eval_V_tau)+int(eval_W_tau)
    eval_G1=multiply(G1, temp)
    eval_aW_tau_G1=multiply(eval_G1, int(a[i]))
    C_0=add(C_0, eval_aW_tau_G1)
C_1=None
C_2=None
C_3=None
C_4=None

In [ ]:

#Le prouver gen r,s deux randoms pour avoir du ZKP
r=GF(30)
s=GF(200)

h_tau=evaluate_polynomial_galois(h_quo.coeffs, tau, GF)
ht_tau=h_tau*t_tau
print("ht(tau)=",ht_tau)

#===============================CALCUL DE A============================================
A_G1=None
A_0=int(alpha)

A_1=0
for i in range(0, nb_eq):
    A_1+=int(a[i])*evaluate_polynomial_galois(U[i], tau, GF)

A_2=int(r)*int(delta)

A_G1=A_0+A_1+A_2
print("A_G1=",A_G1)
print("A_G1=",multiply(G1,A_G1))


#===============================CALCUL DE B_2============================================
B_G2=None
B_0=beta_G2

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G2=multiply(G2, eval_V_tau)
    eval_aV_tau_G2=multiply(eval_V_tau_G2, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G2)

B_2=multiply(delta_G2, int(s))

B_G2=add(B_0, add(B_1, B_2))
print("B_G2=",B_G2)

#===============================CALCUL DE B_1============================================
B_G1=None
B_0=beta_G1

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G1=multiply(G1, eval_V_tau)
    eval_aV_tau_G1=multiply(eval_V_tau_G1, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G1)

B_2=multiply(delta_G1, int(s))

B_G1=add(B_0, add(B_1, B_2))
print("B_G1=",B_G1)

#===============================CALCUL DE C ============================================
C=None

C_0=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_W_tau=evaluate_polynomial_galois(W[i], tau, GF)
    temp=int(beta)*int(eval_U_tau)+int(alpha)*int(eval_V_tau)+int(eval_W_tau)
    eval_G1=multiply(G1, temp)
    eval_aW_tau_G1=multiply(eval_G1, int(a[i]))
    C_0=add(C_0, eval_aW_tau_G1)
C_1=None
C_2=None
C_3=None
C_4=None